# Create function(s) to scrape all racket pages from Tennis Warehouse

In this notebook, I will refactor my code from notebooks 1 and 2 to be more suitable for a single application process.

## Table of Contents
1. [Psuedocode for scraping](#psuedocode-for-scraping)
2. [Get brand URLs](#get-brand-urls)
3. [Get product page URLs](#get-product-page-urls)
4. [Get racquet features](#get-racquet-features)
5. [Scrape entire brand page](#scrape-entire-brand-page)
6. [Scrape all brand pages](#scrape-all-brand-pages)
7. [Export data](#export-data)

## Psuedocode for scraping (+ Imports)

1. Scrape "Shop All" page for URLs of each racket brand and store in a list

2. Iterate over each link in the list:

    - 2.1. Extract product links elements and iterate over each element:
        2.1.1 Extract product page link and store in the list

    - 2.2 Iterate over each product page link in the list:

        - 2.2.1 Extract image link, name, rating, price, and description and store into a dictionary
        - 2.2.2 Extract specs all at once using regex and store in separate dictionary
        - 2.2.3 Combine two dictionaries together using ".update()" or dictionary unpacking
        - 2.2.4 Append dictionary to a pandas dataframe?
        - 2.2.5 Repeat through all product pages

    - 2.3 Return appended pandas df

    - 2.4 Repeat over all brand pages



In [ ]:
# Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

## Get brand URLs
In this section, I create and test a function to get a list of all racquet brand page URLs from the sidebar of the Tennis Warehouse website.

In [ ]:
# Function to get brand URLs given the TW website URL
def get_brand_URLs(shop_all_URL: str) -> list[str]: # Given a string (URL) and outputs a list of strings (URLs)
    
    # Pull and parse webpage html
    webpage = requests.get(shop_all_URL)
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    # Find sidebar links by isolating ul with left_menu-section class
    # and then finding all list items
    sidebar_links = soup.find_all("ul", attrs = {"class": "left_menu-section"})
    brand_elements = sidebar_links[1].find_all("li")
    
    # Initialize emtpy list to store brand extension part of URL
    # then iterate over brand_elements and extract the brand extension
    # to store in brand_pointer_list
    brand_pointer_list = []
    for brand in brand_elements:
        brand_pointer = brand.find("a").get("href")
        brand_pointer_list.append(brand_pointer)
    
    # Create a list of the full brand page URLs by adding the brand 
    # pointer onto the end of the main TW URL
    brand_page_URLs = ["https://www.tennis-warehouse.com"+ pointer for pointer in brand_pointer_list]
    
    return brand_page_URLs # Return list of brand page URLs
    

In [ ]:
# Test get_brand_URLs function - PASSED!
"""
Expected output:
['https://www.tennis-warehouse.com/Babolatracquets.html',
 'https://www.tennis-warehouse.com/Wilsonracquets.html',
 'https://www.tennis-warehouse.com/Headracquets.html',
 'https://www.tennis-warehouse.com/YonexRacquets.html',
 'https://www.tennis-warehouse.com/PrinceRacquets.html',
 'https://www.tennis-warehouse.com/Tecnifibreracquets.html',
 'https://www.tennis-warehouse.com/DunlopRacquets.html',
 'https://www.tennis-warehouse.com/VolklRacquets.html',
 'https://www.tennis-warehouse.com/ProKennexracquets.html',
 'https://www.tennis-warehouse.com/Solinco_Tennis_Racquets/catpage-SOLINCORAC.html',
 'https://www.tennis-warehouse.com/LacosteRacquets.html']
 
"""

brand_page_URLs = get_brand_URLs("https://www.tennis-warehouse.com/TennisRacquets.html")
brand_page_URLs

['https://www.tennis-warehouse.com/Babolatracquets.html',
 'https://www.tennis-warehouse.com/Wilsonracquets.html',
 'https://www.tennis-warehouse.com/Headracquets.html',
 'https://www.tennis-warehouse.com/YonexRacquets.html',
 'https://www.tennis-warehouse.com/PrinceRacquets.html',
 'https://www.tennis-warehouse.com/Tecnifibreracquets.html',
 'https://www.tennis-warehouse.com/DunlopRacquets.html',
 'https://www.tennis-warehouse.com/VolklRacquets.html',
 'https://www.tennis-warehouse.com/ProKennexracquets.html',
 'https://www.tennis-warehouse.com/Solinco_Tennis_Racquets/catpage-SOLINCORAC.html',
 'https://www.tennis-warehouse.com/LacosteRacquets.html']

## Get product page URLs
In this section, I create and test a function to extract a list of each racquet's product URL from a given brand's URL.

In [ ]:
# Function to get product page URLs from brand URL
def get_product_page_URLs(brand_page_URL: str)->list[str]: # Takes a str (brand URL) as input and returns a list of str (product URLs)
    
    # Pull and parse webpage html
    webpage = requests.get(brand_page_URL)
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    # Find product elements by finding all a tags with cattable-wrap-cell-info class
    product_elements = soup.find_all("a", attrs = {"class": "cattable-wrap-cell-info"})
    
    # Initialize an empty list to store product URLs
    product_page_URLs = []
    
    # Iterate over product_elements, extract the product URL, 
    # check if it is a link to a page in TW's website (some listings are just ads), 
    # and add to product_page_URLs
    for product_element in product_elements:
        product_URL = product_element.get("href")
        if "https://www.tennis-warehouse.com/" in product_URL:
            product_page_URLs.append(product_URL)
        else:
            pass
        
    return product_page_URLs # Return list of product page URLs

In [94]:
# Test get_product_page_URLs function - PASSED!
product_page_links = get_product_page_URLs(brand_page_URL= brand_page_URLs[2])
product_page_links

['https://www.tennis-warehouse.com/Head_Speed_Pro_Legend/descpageRCHEAD-HSPDPL.html',
 'https://www.tennis-warehouse.com/Head_Speed_MP_Legend/descpageRCHEAD-HSPMPL.html',
 'https://www.tennis-warehouse.com/Head_Speed_Pro/descpageRCHEAD-HSPDP.html',
 'https://www.tennis-warehouse.com/Head_Speed_MP/descpageRCHEAD-HSPDM.html',
 'https://www.tennis-warehouse.com/Head_Speed_MP_L/descpageRCHEAD-HSPDML.html',
 'https://www.tennis-warehouse.com/Head_Speed_Team/descpageRCHEAD-HSPDTM.html',
 'https://www.tennis-warehouse.com/Head_Speed_Pro/descpageRCHEAD-SPDP.html',
 'https://www.tennis-warehouse.com/Head_Speed_MP/descpageRCHEAD-SPDM.html',
 'https://www.tennis-warehouse.com/Head_Graphene_XT_Speed_MP/descpageRCHEAD-GXSMPR.html',
 'https://www.tennis-warehouse.com/Head_Graphene_XT_Speed_S/descpageRCHEAD-GXSS.html',
 'https://www.tennis-warehouse.com/Head_Boom_Pro/descpageRCHEAD-HBOOMP.html',
 'https://www.tennis-warehouse.com/Head_Boom_MP/descpageRCHEAD-HBOOMM.html',
 'https://www.tennis-warehous

## Get racquet features
In this section, I create and test a function to create a data frame of each racquet's features from a given racquet's URL.

In [ ]:
# Pick a racquet page to use for testing
product_page_links[9]

'https://www.tennis-warehouse.com/Head_Graphene_XT_Speed_S/descpageRCHEAD-GXSS.html'

In [ ]:
# Function to extract all features of a racquet given a URL to its product page
def get_racquet_features(product_page_URL: str) -> pd.DataFrame: # Takes a str input (URL) and returns a pandas df of racquet features
    
    # Pull and parse racquet page html
    webpage = requests.get(product_page_URL)
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    # EXTRACT FEATURES FROM TOP PART OF PAGE
    
    # Initialize an empty dict to store racquet info
    racquet_info = {}
    
    # Extract racquet image linnk and name by using img and h1 tags and associated classes
    racquet_info["racquet_img"] = soup.find("img", attrs = {"class": "main_image is-zoomable"}).get("src") # type: ignore
    racquet_info["racquet_name"] = soup.find("h1", attrs = {"class": "h2 desc_top-head-title"}).text # type: ignore
    
    # Check if the racket has ratings, if not assign the rating as NA
    if soup.find("div", attrs = {"class": "review_agg"}):
        racquet_info["racquet_rating"] = float(soup.find("div", attrs = {"class": "review_agg"}).text) # type: ignore
    else:
        racquet_info["racquet_rating"] = np.nan

    ## The code below doesn't work because it throws an error when a racket has no ratings
    #racquet_info["racquet_rating"] = float(soup.find("div", attrs = {"class": "review_agg"}).text)
    
    # Extract racquet price and racquet description by using span and div tags with associated classes
    racquet_info["racquet_price"] = float(soup.find("span", attrs = {"class": "afterpay-full_price"}).text) # type: ignore
    racquet_info["racquet_desc"] = soup.find("div", attrs = {"class": "check_read-inner"}).text # type: ignore
    
    ## The if statment below doesn't work because there was more than one inconsistency with the description tagging
    ## - kept for documenting problem solving process
    #if soup.find("p", attrs = {"style": "text-align: justify;"}):
    #    racquet_info["racquet_desc"] = soup.find("p", attrs = {"style": "text-align: justify;"}).text
    #else:
    #    racquet_info["racquet_desc"] = soup.find("div", attrs = {"style": "text-align: justify;"}).text
    
    ## The code below doesn't work because not all racket descriptions are written between p tags
    ## - kept for documenting problem solving process
    #racquet_info["racquet_desc"] = soup.find("p", attrs = {"style": "text-align: justify;"}).text
    
    # EXTRACT FEATURES FROM BOTTOM SPEC TABLE OF PAGE
    
    # Initialize an empty dict to store spec values from bottom of racquet page
    racquet_specs = {}
    
    # Check if the racquet page has a spec table, if not create a dict with all NaN values
    if soup.find("tbody"):
        # Extract all spec rows
        racquet_spec_elements = soup.find("tbody").find_all("td", class_=re.compile("Specs")) # type:ignore
        
        # Iterate over racquet_spec elements, pull out bolded texts 
        # as column name and value after ":" as row value
        for spec in racquet_spec_elements:
            if spec.find("strong"):
                label = spec.find("strong").text.split(":")[0].strip()
                value = spec.text.split(":")[1].strip()
            else: # If there isn't a bold text, record it in a column labeled "other"
                label = "Other"
                value = spec.text.strip()
            
            ## The code below doesn't work becuase it doesn't account for pages without any specs 
            ## - kept for documenting problem solving process
            #label = spec.find("strong").text.split(":")[0].strip()
            #value = spec.text.split(":")[1].strip()
            
            # Add label and value to empty dict
            racquet_specs[label] = value
    else:
        racquet_specs = {"Head Size": np.nan,
                  "Length": np.nan,
                  "Strung Weight": np.nan,
                  'Balance:': np.nan,
                  'Swingweight:': np.nan,
                  'Stiffness:': np.nan,
                  'Beam Width:': np.nan,
                  'Composition:': np.nan,
                  'Power Level:': np.nan,
                  'Stroke Style:': np.nan,
                  'Swing Speed:': np.nan,
                  'Racquet Colors:': np.nan,
                  'Grip Type:': np.nan,
                  'String Pattern:': np.nan,
                  'String Tension:': np.nan}
    
    # Combine top info and specs dictionaries and turn into a df
    racquet_info.update(racquet_specs)
    racquet_info_df = pd.DataFrame(racquet_info, index=[0])
    
    return racquet_info_df 

In [99]:
# Test get_racquet_features function - PASSED!
racquet_info_df = get_racquet_features(product_page_URL=product_page_links[9])
racquet_info_df

,racquet_img,racquet_name,racquet_rating,racquet_price,racquet_desc,Head Size,Length,Strung Weight,Balance,Swingweight,...,Beam Width,Composition,Power Level,Stroke Style,Swing Speed,Racquet Colors,Grip Type,String Pattern,String Tension,Other
0,https://img.tennis-warehouse.com/watermark/rs....,Head Graphene XT Speed S,5.0,109.0,Pre-strung with a synthetic gut for added con...,100 in² / 645 cm²,27in / 68.5cm,10.7oz / 303.34g,13.35in / 33.91cm / 1 pts HL,317,...,22.5mm / 22.5mm / 22mm /,Graphene XT/Graphite,Low-Medium,Medium-Full,Medium-Fast,Black/ White,Hydrosorb Pro,16 Mains / 19 CrossesMains skip,48-57 lbs / 22-26 kg,Sony Smart Tennis Sensor Ready


## Scrape entire brand page
In this section, I create and test a function to extract a data frame all racquets' features on a brand page given the brand's URL.

In [ ]:
# Function that takes in a brand page's link and returns a data frame of features of each racket listed in that brand's page

# I was getting some AttributeErrors with this function so I implemented some logging messages. 
# The issue turned out to be formatting inconsistencies in the product page. The appropriate changes 
# were made to the get_racquet_features() function to fix the errors. 

# Logging module to check what error I was getting
import logging
logging.basicConfig(level=logging.INFO) # Set default logging level

# Define function
def scrape_brand_page(brand_page_URL: str) -> pd.DataFrame: # Takes a brand page URL and returns a df of ALL racquets and their features
    
    # Extract a list of product URLs from brand URL 
    # -> uses get_product_page_URLs function from above
    product_URLs = get_product_page_URLs(brand_page_URL= brand_page_URL)
    
    # Initialize an empty data frame to append each racquet's info onto
    total_racquet_info_df = pd.DataFrame()
    
    ## Initialize a counter for logging purposees
    # i = 0
    
    # Iterate over each product URL, extract the product (racquet)'s features 
    # and store it into a temporary df, then concatenate the temp df to the total_racquet_info_df
    for product_URL in product_URLs:
        racquet_info_df = get_racquet_features(product_URL)
        total_racquet_info_df = pd.concat([total_racquet_info_df, racquet_info_df])
        ## Increment counter by 1
        # i += 1
        
        ## Logging messages when I was trying to debug this function
        ## - kept for documenting problem solving process
        # print(f"Completed racket{i}")
        # logging.debug(f"Completed racket {i}")
        
    ## Logging messages when I was trying to debug this function
    ## - kept for documenting problem solving process
    # logging.info(f"Successfully scraped all {i} rackets (whew)!")
    
    return total_racquet_info_df



In [ ]:
# Test to get final product page df - PASSED!
total_r_df = scrape_brand_page(brand_page_URLs[2])

Completed racket0
Completed racket1
Completed racket2
Completed racket3
Completed racket4
Completed racket5
Completed racket6
Completed racket7
Completed racket8
Completed racket9
Completed racket10
Completed racket11
Completed racket12
Completed racket13
Completed racket14
Completed racket15
Completed racket16
Completed racket17
Completed racket18
Completed racket19
Completed racket20
Completed racket21
Completed racket22
Completed racket23
Completed racket24
Completed racket25
Completed racket26
Completed racket27
Completed racket28
Completed racket29
Completed racket30
Completed racket31
Completed racket32
Completed racket33
Completed racket34
Completed racket35
Completed racket36
Completed racket37
Completed racket38
Completed racket39
Completed racket40
Completed racket41
Completed racket42
Completed racket43
Completed racket44
Completed racket45
Completed racket46
Completed racket47
Completed racket48
Completed racket49
Completed racket50
Completed racket51
Completed racket52
Com

INFO:root:Successfully scraped all 68 rackets (whew)!


Completed racket67


In [106]:
#Check if scrape_brand_page works for all brands -- 
# catch any weird formatting errors
# LET'S GOOOO IT FINALLY KINDA WORKS

brand_dict = {"Babolat": brand_page_URLs[0],
              "Wilson": brand_page_URLs[1],
              "Head": brand_page_URLs[2],
              "Yonex": brand_page_URLs[3],
              "Prince": brand_page_URLs[4],
              "Tecnifibre": brand_page_URLs[5],
              "Dunlop": brand_page_URLs[6],
              "Volkl": brand_page_URLs[7],
              "ProKennex": brand_page_URLs[8],
              "Solinco": brand_page_URLs[9],
              "Lacoste": brand_page_URLs[10]}


for brand_name, brand_url in brand_dict.items():
    test = scrape_brand_page(brand_url)
    print(f"Successfully scraped {brand_name}")

Successfully scraped Babolat
Successfully scraped Wilson
Successfully scraped Head
Successfully scraped Yonex
Successfully scraped Prince
Successfully scraped Tecnifibre
Successfully scraped Dunlop
Successfully scraped Volkl
Successfully scraped ProKennex
Successfully scraped Solinco
Successfully scraped Lacoste


In [86]:
total_r_df

,racquet_img,racquet_name,racquet_rating,racquet_price,racquet_desc,Head Size,Length,Strung Weight,Balance,Swingweight,...,Power Level:,Stroke Style:,Swing Speed:,Racquet Colors:,Grip Type:,String Pattern:,String Tension:,Age,Weight,Height
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 2025,4.8,289.0,The Pure Drive is popular for a reason. Boast...,100 in² / 645.16 cm²,27in / 68.58cm,11.2oz / 318g,12.99in / 32.99cm / 4 pts HL,317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2025,4.5,299.0,Originally launched in 2019 under the VS moni...,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,326,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2-Pack 2025,5.0,579.0,This product is for 2 Pure Drive 98 racquets....,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,323,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Plus 2025,5.0,289.0,Babolat adds another chapter to one of the ga...,100 in² / 645.16 cm²,27.5in / 69.85cm,11.2oz / 318g,13in / 33.02cm / 6 pts HL,325,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Team 2025,5.0,269.0,The Pure Drive Team 2025 is defined by its us...,100 in² / 645.16 cm²,27in / 68.58cm,10.6oz / 301g,12.85in / 32.64cm / 5 pts HL,308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,https://img.tennis-warehouse.com/watermark/rs....,"Babolat B Fly 23"" 2023 Junior",NaN,44.0,"Babolat updates the B Fly 23"" with an eye-cat...",96 in² / 619.35 cm²,23 in / 58.42 cm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6-8,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,"Babolat B Fly 25"" 2023 Junior",NaN,44.0,"Babolat updates the B Fly 25"" with an eye-cat...",106 in² / 683.87 cm²,25 in / 63.50 cm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9-10,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,"Babolat B Fly 21"" 2023 Junior",NaN,39.0,Updated with an awesome new cosmetic for 2023...,95 in² / 612.90 cm²,21 in / 53.34 cm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-5,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,"Babolat B Fly 19"" 2023 Junior",5.0,39.0,"Updated with a new cosmetic for 2023, the B F...",81 in² / 522.58 cm²,19 in / 48.26 cm,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-5,NaN,NaN


## Scrape all brand pages
In this section, I create and test a function to extract a data frame of ALL racquets and their features from TW's website given a list of each brand page's URLs.

In [ ]:
# Function to create data frames for all rackets on each brand's page and then combine all of the data frames into one.
# logging.basicConfig(level=logging.DEBUG) # Set default logging level

# Define function -> Takes a list of str (brand page URLs) and returns a df
def scrape_all_brand_pages(brand_page_URLs: list[str]) -> pd.DataFrame:
    ## Initialize a counter for logging
    # i = 0
    
    # Initialize an empty df to hold ALL racquets
    final_df = pd.DataFrame()
    
    # Iterate over each brand page, scrape the brand page and store 
    # the racquet features in a temp df, then concatenate the temp df to the final_df
    for brand_URL in brand_page_URLs:
        df = scrape_brand_page(brand_URL)
        final_df = pd.concat([final_df, df])
        
        ## Logging messages for debugging
        # logging.debug(f"Completed brand {i}")
        # i += 1
    
    ## Final logging success message
    #logging.info(f"Successfully scraped all {i} brands (if this works on the first try it'd be a miracle)!")
    
    return final_df

In [ ]:
# Test function by scraping all brand pages into one data frame
#logging.basicConfig(level=logging.DEBUG) # Set default logging level

all_brands_df = scrape_all_brand_pages(brand_page_URLs=brand_page_URLs)

INFO:root:Successfully scraped all 11 brands (if this works on the first try it'd be a miracle)!


In [ ]:
# View all_brands_df
all_brands_df

,racquet_img,racquet_name,racquet_rating,racquet_price,racquet_desc,Head Size,Length,Strung Weight,Balance,Swingweight,...,Swing Speed:,Racquet Colors:,Grip Type:,String Pattern:,String Tension:,Age,Weight,Height,Other,Strung Weight
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 2025,4.8,289.00,The Pure Drive is popular for a reason. Boast...,100 in² / 645.16 cm²,27in / 68.58cm,11.2oz / 318g,12.99in / 32.99cm / 4 pts HL,317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2025,4.5,299.00,Originally launched in 2019 under the VS moni...,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,326,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2-Pack 2025,5.0,579.00,This product is for 2 Pure Drive 98 racquets....,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,323,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Plus 2025,5.0,289.00,Babolat adds another chapter to one of the ga...,100 in² / 645.16 cm²,27.5in / 69.85cm,11.2oz / 318g,13in / 33.02cm / 6 pts HL,325,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Team 2025,5.0,269.00,The Pure Drive Team 2025 is defined by its us...,100 in² / 645.16 cm²,27in / 68.58cm,10.6oz / 301g,12.85in / 32.64cm / 5 pts HL,308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,https://img.tennis-warehouse.com/watermark/rs....,Solinco Blackout 300 XTD,4.8,229.99,"With the Blackout 300 XTD, Solinco takes the ...",100 in² / 645.16 cm²,27.5in / 69.85cm,11.3oz / 320g,12.8in / 32.51cm / 8 pts HL,328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Solinco Blackout 300 XTD+,5.0,229.99,"With the Blackout 300 XTD+, Solinco gives adv...",100 in² / 645.16 cm²,28in / 71.12cm,11.3oz / 320g,12.8in / 32.51cm / 10 pts HL,333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Lacoste L23,4.5,199.00,Introducing the Lascoste L23! Following on th...,100 in² / 645.16 cm²,27in / 68.58cm,11.1oz / 315g,12.9in / 32.77cm / 5 pts HL,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Lacoste L23L,5.0,199.00,Lacoste makes impressive updates to the L23L ...,100 in² / 645.16 cm²,27in / 68.58cm,10.2oz / 289g,13.4in / 34.04cm / 1 pts HL,310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stiffness: 68,NaN


## Export data
In this section, I export the generated df as a csv.

In [ ]:
# Write df to csv file for reproducibility and ease of access
all_brands_df.to_csv("../racquet_features_raw.csv", sep = ",")